__Inference set up - Basemodel_FMD__

In [1]:
from fastai.vision.all import *
import os
import pathlib
import fastai; fastai.__version__

'2.7.15'

In [2]:
torch.cuda.set_device(0)

In [3]:
torch.cuda.current_device(), torch.cuda.get_device_name(0)

(0, 'NVIDIA GeForce RTX 4060 Ti')

In [4]:
device=torch.device("cuda")

In [5]:
path = pathlib.Path(os.getcwd())

In [6]:
path_GT = path/'Daten/FMD_GT'
path_noisy = path/'Daten/FMD_noisy'

In [7]:
set_seed(42)

In [8]:
%run Dataloader_FMD.ipynb
%run Perception_loss.ipynb

/home/david/mambaforge/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/home/david/mambaforge/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
dls_den = get_dls(38, 256)  # required to generate learner       

In [10]:
loss_func = FeatureLoss(vgg_m, blocks[2:5],[1], [1,1,1], [1,1,1]) # dummy

In [11]:
# define learner w/o learning parameters (e.g. metrics, cbs)
def create_gen_learner():
    return unet_learner(dls_den, bbone, loss_func=loss_func, blur=True, norm_type=NormType.Weight, 
                        self_attention=True, y_range=y_range
                        )

In [12]:
# create learner
learn_den = create_gen_learner()
# load the Basemodel into that leaner and convert to fp16
learn_den.load('Basemodel_FMD').to_fp16

/home/david/mambaforge/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


<bound method Learner.to_fp16 of <fastai.learner.Learner object at 0x7c9950d112a0>>

__Set up data and perform inference__

In [13]:
path_test = path/'Daten/FMD_testmix/noisy_tiles'
path_preds = path/'Daten/FMD_testmix/FMD_pred_tiles'
path_preds.mkdir(exist_ok=True)

In [14]:
# check path
test_files = get_image_files(path_test)
test_files[0], len(test_files)

(Path('/home/david/mnt_anarchyNAS/lohrd/Github/AIMD/Daten/FMD_testmix/noisy_tiles/WideField_BPAE_G_1avg16_tile_2.png'),
 960)

In [15]:
# create test set dataloader
test_dl = learn_den.dls.test_dl(test_files)

In [16]:
# generate predictions, decode oop over all predictions and save them as 8 bit png
def save_preds(dl, learn):
  "Save away predictions"
  names = dl.dataset.items
  
  inp, preds, _,decoded = learn.get_preds(dl=dl, with_input=True, with_decoded=True)
  for i,pred in enumerate(preds):
      dec = dl.after_batch.decode((TensorImage(pred[None]),))[0][0]
      arr = dec.numpy().transpose(1,2,0).astype(np.uint8)
      Image.fromarray(arr[:,:,0]).save(path_preds/names[i].name)
  return inp, preds, decoded       # return outputs for analysis

In [17]:
inp, preds, dec = save_preds(test_dl, learn_den)

/home/david/mambaforge/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1716905971132/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:84.)
  return F.conv2d(input, weight, bias, self.stride,


In [18]:
# use this loop with a chunksize that works for your RAM, if your RAM cant contain all predictions simultaneously
"""
chunk_size = 5000
for k in range(0,len(test_files),chunk_size):
    chunk = test_files[k:k+chunk_size]
    test_dl=learn_den.dls.test_dl(chunk)
    inp, preds, dec = save_preds(test_dl, learn_den)
"""

'\nchunk_size = 5000\nfor k in range(0,len(test_files),chunk_size):\n    chunk = test_files[k:k+chunk_size]\n    test_dl=learn_den.dls.test_dl(chunk)\n    inp, preds, dec = save_preds(test_dl, learn_den)\n'